In [2]:
import numpy as np
class price_node:
    def __init__(self, step, S ,K,cp='call',ae='European'):
        self.step = step
        self.S = S
        self.value = max(S-K,0) if cp=='call' else max(K-S,0)
        self.left = None
        self.right = None    
    def __repr__(self):
        return str('step:'+str(self.step)+'     S:'+str(self.S)+'   value:'+str(self.value))
    
class binomial_tree:
    def __init__(self, sigma = 0.1, r = 0.05, S0 = 10, K = 12, T = 3, dt =0.5,cp='call',ae='European'):
        self.n=int(T/dt)#terminal step
        self.root = price_node(0,S0,K)
        self.r=r;self.sigma=sigma
        self.S0=S0;self.K=K
        self.T=T;self.dt=dt
        self.u=np.exp(sigma*dt**0.5) ;self.d=np.exp(-sigma*dt**0.5)
        self.p=(np.exp(r*dt)-self.d)/(self.u-self.d)
        self.value=0
        self.n=int(T/dt)
        self.cp=cp
        self.ae=ae
        self.generate_tree(self.root)
    
    def generate_tree(self,node):
        if node.step==self.n:
            node.value=max(node.S-self.K,0) if self.cp=='call' else max(self.K-node.S,0)
        else:
            node.left=price_node(node.step+1,node.S*self.u,self.K,cp=self.cp,ae=self.ae)
            node.right=price_node(node.step+1,node.S*self.d,self.K,cp=self.cp,ae=self.ae)
            self.generate_tree(node.left)
            self.generate_tree(node.right)
            if self.ae=='European':
                node.value=np.exp(-self.r*self.dt)*(self.p*node.left.value+(1-self.p)*node.right.value)
            else:
                node.value=max(np.exp(-self.r*self.dt)*(self.p*node.left.value+(1-self.p)*node.right.value), node.value)    

    def print_tree(self,node):
        if node is None:
            return
        print(node)
        self.print_tree(node.left)
        self.print_tree(node.right)



In [3]:
tree=binomial_tree(S0=29,K=30,T=1,dt=0.5,sigma=0.25,r=0.03,cp='put',ae='European')   
tree.print_tree(tree.root)

step:0     S:29   value:2.83771598784725
step:1     S:34.60757280399054   value:0.4940878867328611
step:2     S:41.29945156494848   value:0
step:2     S:29.000000000000004   value:0.9999999999999964
step:1     S:24.301039681783916   value:5.252318506307964
step:2     S:29.0   value:1.0
step:2     S:20.363466538470227   value:9.636533461529773


In [4]:
tree=binomial_tree(S0=29,K=30,T=1,dt=0.5,sigma=0.25,r=0.03,cp='put',ae='American')   
tree.print_tree(tree.root)

step:0     S:29   value:3.05839629681947
step:1     S:34.60757280399054   value:0.4940878867328611
step:2     S:41.29945156494848   value:0
step:2     S:29.000000000000004   value:0.9999999999999964
step:1     S:24.301039681783916   value:5.698960318216084
step:2     S:29.0   value:1.0
step:2     S:20.363466538470227   value:9.636533461529773


Cheuk-Vorst is another way to calculate American option price, and it only uses half of the tree.

Trinomial schemes:
$$V=\frac{p_1V_u^{\Delta t}+p_2V_u^{\Delta t}+p_3V_u^{\Delta t}}{e^{r\Delta t}}$$
$$p1=\frac{1}{\lambda^2}+\frac{(r-\frac{\sigma^2}{2}) \sqrt{\Delta t}}{2\lambda\sigma}$$
$$p2=1-\frac{1}{\lambda^2}$$
$$p3=\frac{1}{\lambda^2}-\frac{(r-\frac{\sigma^2}{2}) \sqrt{\Delta t}}{2\lambda\sigma}$$

our experience is that $\lambda=\sqrt{3}$ is the best

Multistate extension- Kamrad-Ritchken's Approach

$$p1(s1u,s2u)=\frac{1}{4}+\frac{\sqrt{\Delta t}}{\lambda}(\frac{r-\frac{\sigma^2_1}{2}}{\sigma_1}+\frac{r-\frac{\sigma^2_2}{2}}{\sigma_2})$$
$$p2(s1u,s2d)=\frac{1}{4}+\frac{\sqrt{\Delta t}}{\lambda}(\frac{r-\frac{\sigma^2_1}{2}}{\sigma_1}-\frac{r-\frac{\sigma^2_2}{2}}{\sigma_2})$$
$$p3(s1d,s2d)=\frac{1}{4}+\frac{\sqrt{\Delta t}}{\lambda}(-\frac{r-\frac{\sigma^2_1}{2}}{\sigma_1}-\frac{r-\frac{\sigma^2_2}{2}}{\sigma_2})$$
$$p4(s1d,s2u)=\frac{1}{4}+\frac{\sqrt{\Delta t}}{\lambda}(-\frac{r-\frac{\sigma^2_1}{2}}{\sigma_1}+\frac{r-\frac{\sigma^2_2}{2}}{\sigma_2})$$
$$p5(s1h,s2h)=1-\frac{1}{\lambda^2}$$

Forward shooting grid methods(path dependent options)

The approach of appending au auxiliary state vector at each node in the lattice tree to model the correlated evolution of $F_t$ with $S_t$ is commonly called the FSG method. path dependent function:
$$F_{t+\Delta t}=G(F_t,t,S_{t+\Delta t})$$

Let $g(k,n,j) denote the grid function which is considered as the discrete analog of the evolution function G, here K is the index for $F_t$,n is the index for t and j is the index for $S_{t+\Delta t}$

The trinomial version of FSG scheme can be represented as:
$$V^n_{j,k}=[p_uV^{n+1}_{j+1,g(k,n,j+1)} + p_0V^{n+1}_{j,g(k,n,j)}+ p_dV^{n+1}_{j-1,g(k,n,j-1)}]e^{-r \Delta t}$$